In [269]:
import gensim
import pandas as pd
import pickle

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
#initialize tqdm
tqdm.pandas()

In [94]:
len(topic_titles)

82

In [252]:
DATA_DIR = '../../data/reddit/Article_data_2019/'
useful_topics = [2,4,6,7,8,9,12,13,14,17,18,19,20,21,22,23,24,26,28,29,30,31,32,33,34,35,38,42,44,45,46,47,48,50,51,
                 52,53,55,58,61,62,63,64,66,67,68,69,70,71,75,77,78,79,80,85,86,87,88,89,91,92,93,96,99]

topic_titles = ['2020 election', 'trump administration', 'climate change', 'trump foreign policy', 'mueller report', 
                'cohen', 'williamson', '2018 congress', 'border', 'first debate', 'hunder biden', 'town hall', 
                'first debate', 'biden busing', 'female congresswomen', 'nxivm sex cult', 'auto industry', 
                'assange + stone', 'healthcare', 'de blasio', 'buttigieg', 'impeachment', 
                'us economy (socialism vs captialism)', 'economic foreign policy', 'iowa caucus', 
                'north carolina voter fraud', 'north korea', 'jean carroll', '2020 election', 'kavanaugh', 
                'harris', 'mccain', 'aoc', 'sanders', 'border', 'prison reform', 'getty', 'graham', 
                '2020 democractic party', 'stephanopoulos', 'tax fact check', 'generic words', 'o\'rourke', 
                'media words', 'drugs', 'venezuela', 'tech', 'iran', 'assange', 'random', 'healthcare', 
                'harris', 'student debt', 'russian interference', 'politico', 'booker', 'socialism', 
                'megan davis', 'lgbtq', 'swalwell', 'former counsel for mueller report', 'trump racist', 
                'sanders', 'sanders', 'puerto rico', 'reuters', 'hickenlooper', 'tax return', 'supreme court', 
                '2020 democratic primary', 'electoral college', 'inslee', 'gravel', 'generic words', 'socialism', 
                'marijuana', 'maher', 'gun laws', 'yang', 'unions', '2020 democractic primary', 'media words', 
                'hamilton musical??', 'pundits', 'stacey abrams', 'trump', 'warren', 'israel', 'congress', 'harris', 
                'southern racism', 'civil rights', 'busing', 'north korea', 'gold', 'stufff', 'mcconnell', 'stuffs', 
                'generic words', 'iran/afghanistan']

candidates = ["SANDERS","DELANEY","WARREN","HARRIS","GILLIBRAND","O'ROURKE","KLOBUCHAR","BOOKER",
    "BUTTIGIEG","GABBARD","YANG","INSLEE","HICKENLOOPER","WILLIAMSON","TULSI","CASTRO","BIDEN", "BERNIE",
    "BETO", "ROURKE", "BENNETT", "BULLOCK", "BLASIO", "TIM RYAN", "GRAVEL"]
candidates = set([x.lower() for x in candidates])

candidate_dict = {'klobuchar': 'klobuchar',
 'bennett': 'bennett',
 'booker': 'booker',
 'warren': 'warren',
 'castro': 'castro',
 'williamson': 'williamson',
 'gabbard': 'gabbard',
 'bernie': 'sanders',
 "o'rourke": "o_rourke",
 'bullock': 'bullock',
 'tim ryan': 'tim_ryan',
 'sanders': 'sanders',
 'biden': 'biden',
 'hickenlooper': 'hickenlooper',
 'blasio': 'de_blasio',
 'yang': 'yang',
 'delaney': 'delaney',
 'gillibrand': 'gillibrand',
 'beto': 'o_rourke',
 'harris': 'harris',
 'inslee': 'inslee',
 'rourke': 'o_rourke',
 'gravel': 'gravel',
 'tulsi': 'gabbard',
 'buttigieg': 'buttigieg'}

In [60]:
# source data
filename = DATA_DIR + 'LDA/df.pkl'
df = pd.read_pickle(filename)

In [14]:
# model and corpus
model = gensim.models.LdaModel.load(DATA_DIR + '/LDA/models/100_model_10_article_text_2019_candidate_only_len4.gensim')
dictionary = gensim.corpora.Dictionary.load(DATA_DIR + 'LDA/article_text_2019_candidate_only_dictionary.gensim')
corpus = pickle.load(open(DATA_DIR + 'LDA/article_text_2019_candidate_only' + '_corpus.pkl', 'rb'))

/usr/local/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [72]:
df['corpus'] = df.text_lemmas.progress_map(lambda x: dictionary.doc2bow(x))
df['topics'] = df.corpus.progress_map(lambda x: list(filter(lambda y: y[0] in useful_topics,model[x])))

In [101]:
def vector_concatenator(vector):
    v = [(topic_titles[y[0]], y[1]) for y in vector]
    v_final = dict()
    for t in v:
        if t[0] in v_final.keys():
            v_final[t[0]] += t[1]
        else:
            v_final[t[0]] = t[1]
    return list(zip(v_final.keys(), v_final.values()))

df['topic_titles'] = df.topics.progress_map(vector_concatenator)

In [231]:
def intersection(lemma): 
    lemma = [token.lower() for token in lemma]
    intersect = candidates.intersection(lemma)
    return {candidate_dict[x] for x in intersect}

df['candidate_title'] = df['title_lemmas'].progress_map(intersection)
df['candidate_text'] = df['text_lemmas'].progress_map(intersection)

In [233]:
df.to_csv(DATA_DIR + 'final_top_topics_news_articles.csv')

### Aggregated in Time Series

In [176]:
### get date dataframes
df_date = pd.read_pickle(DATA_DIR + 'reddit_2019jun16tojul1_dates.pkl')
df_date = df_date.append(pd.read_pickle(DATA_DIR + 'reddit_2019_dates.pkl'))
df_date = df_date.append(pd.read_csv(DATA_DIR + 'dates_missing_ids.csv', index_col=0))

In [262]:
df2 = df.merge(df_date, on='id', how='left') #df.join(df_date, on='id', how='left')
df2['day'] = pd.to_datetime(df2['created_utc'], yearfirst=True).dt.round('d')
df2['candidate_text'] = df2.candidate_text.map(lambda x: list(x))
df2['counter'] = 1
df2.counter = df2.counter.astype('object')

In [345]:
def vector_concatenator(row):
    vector = row.topic_titles
    v_final = dict()
    for t in vector:
        if t[0] in v_final.keys():
            v_final[t[0]] += t[1]
        else:
            v_final[t[0]] = t[1]
    return list(zip(v_final.keys(), map(lambda x: x/row.counter, v_final.values())))

def vector_counter(row):
    vector = row.candidate_text
    v_final = dict()
    for t in vector:
        if t in v_final.keys():
            v_final[t] += 1
        else:
            v_final[t] = 1
    return list(zip(v_final.keys(), map(lambda x: x/row.counter, v_final.values())))

def normalize_topics(row):
    tt = row.topic_titles.copy()
    s = 0
    for x in tt:
        s += x[1]
        
    for i in range(len(tt)):
        tt[i] = (tt[i][0], tt[i][1]*(1.0/s))
        
    return tt

candidate_dfs = dict()
for candidate in set(candidate_dict.values()):
    candidate_dfs[candidate] = df2[df2.candidate_text.map(lambda x: candidate in x)]\
                                [['counter', 'score', 'candidate_text', 'topic_titles','day']]\
                                .groupby('day').agg(sum)
    candidate_dfs[candidate].topic_titles = candidate_dfs[candidate].apply(vector_concatenator, axis=1)
    candidate_dfs[candidate].candidate_text = candidate_dfs[candidate].apply(vector_counter, axis=1)
    candidate_dfs[candidate].topic_titles = candidate_dfs[candidate].apply(normalize_topics, axis=1)

In [351]:
!mkdir {DATA_DIR + 'candidate_aggregation/'}
for candidate in set(candidate_dict.values()):
    candidate_dfs[candidate].to_csv(DATA_DIR + 'candidate_aggregation/' + candidate + '.csv')

mkdir: ../../data/reddit/Article_data_2019/candidate_aggregation/: File exists


In [348]:
candidate_dfs['harris'].topic_titles[50]

[('first debate', 0.025658442230806204),
 ('sanders', 0.12106575735380976),
 ("o'rourke", 0.00997618037889153),
 ('tech', 0.012727913236780393),
 ('assange', 0.0015186678095882476),
 ('healthcare', 0.09580821438594371),
 ('2020 democratic primary', 0.01164144028923045),
 ('trump', 0.007296036235960234),
 ('harris', 0.07515869516681667),
 ('civil rights', 0.02431009758770042),
 ('climate change', 0.09689594971142895),
 ('2018 congress', 0.04451531920373167),
 ('aoc', 0.010611854272047413),
 ('border', 0.01394964301242738),
 ('economic foreign policy', 0.0032955526178973294),
 ('kavanaugh', 0.015168810119879413),
 ('mccain', 0.018358680340484972),
 ('supreme court', 0.016149511781106827),
 ('congress', 0.060144394163205064),
 ('2020 democractic party', 0.03043741572403697),
 ('iran', 0.002539798609946921),
 ('marijuana', 0.07148023242772159),
 ('gun laws', 0.0015381450100615947),
 ('unions', 0.001843690166288994),
 ('warren', 0.010924470076767136),
 ('mueller report', 0.00119814433935030

In [350]:
l = [('first debate', 0.025658442230806204),
 ('sanders', 0.12106575735380976),
 ("o'rourke", 0.00997618037889153),
 ('tech', 0.012727913236780393),
 ('assange', 0.0015186678095882476),
 ('healthcare', 0.09580821438594371),
 ('2020 democratic primary', 0.01164144028923045),
 ('trump', 0.007296036235960234),
 ('harris', 0.07515869516681667),
 ('civil rights', 0.02431009758770042),
 ('climate change', 0.09689594971142895),
 ('2018 congress', 0.04451531920373167),
 ('aoc', 0.010611854272047413),
 ('border', 0.01394964301242738),
 ('economic foreign policy', 0.0032955526178973294),
 ('kavanaugh', 0.015168810119879413),
 ('mccain', 0.018358680340484972),
 ('supreme court', 0.016149511781106827),
 ('congress', 0.060144394163205064),
 ('2020 democractic party', 0.03043741572403697),
 ('iran', 0.002539798609946921),
 ('marijuana', 0.07148023242772159),
 ('gun laws', 0.0015381450100615947),
 ('unions', 0.001843690166288994),
 ('warren', 0.010924470076767136),
 ('mueller report', 0.0011981443393503097),
 ('2020 election', 0.050558334675905134),
 ('prison reform', 0.014341456638068581),
 ('booker', 0.005320117616564274),
 ('trump racist', 0.03739508139115104),
 ('electoral college', 0.019965296032990233),
 ('williamson', 0.006652046735552784),
 ('de blasio', 0.0009333922037186751),
 ('2020 democractic primary', 0.008768565259136399),
 ('us economy (socialism vs captialism)', 0.014495256691405192),
 ('student debt', 0.04034176249484875),
 ('hickenlooper', 0.0020900745492633908),
 ('female congresswomen', 0.0013331784325417689),
 ('north korea', 0.008957454705650255),
 ('israel', 0.0027223592599369276),
 ('mcconnell', 0.001046901446675053),
 ('iran/afghanistan', 0.0008656656146815745)]

s = 0
for x in l:
    s += x[1]
s

1.0